In [1]:
#1. 모듈 호출
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
#2. 사용자 인터페이스 만들기 
print('='*100)
print('다음 뉴스기사 댓글 크롤러')
print('='*100)
url = input('1.댓글을 크롤링할 뉴스 URL을 입력하세요 : ')
key_word2 = '다음 뉴스 기사 댓글자료 '
cnt = int(input('2.크롤링 할 건수는 몇건입니까? : '))
f_dir = input('3.파일을 저장할 폴더명만 쓰세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word2)
os.chdir(f_dir+s+'-'+key_word2)

f_name = s+'-'+key_word2+'.txt'
fc_name = s+'-'+key_word2+'.csv'
fx_name = s+'-'+key_word2+'.xlsx'

s_time = time.time()

다음 뉴스기사 댓글 크롤러
1.댓글을 크롤링할 뉴스 URL을 입력하세요 : https://news.v.daum.net/v/20210117091900066
2.크롤링 할 건수는 몇건입니까? : 100
3.파일을 저장할 폴더명만 쓰세요 : 


In [3]:
#3. 웹드라이버 호출 
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

driver = webdriver.Chrome(path, options=options)
driver.get(url)
time.sleep(random.randrange(2,5))

In [4]:
driver.find_element_by_id('alexCounter').click()
time.sleep(3)

    
driver.find_element_by_xpath('/html/body/div[2]/div[4]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/ul[1]/li[2]/button/span/span').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[2]/div[4]/div[2]/div[1]/div[2]/div[6]/div/div/div/div/div[3]/div[1]/button/span').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[2]/div[4]/div[2]/div[1]/div[2]/div[6]/div/div/div/div[2]/div[2]/div/div[2]/dl/dd/button/span').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[2]/div[4]/div[2]/div[1]/div[2]/div[6]/div/div/div/div[2]/div[2]/div/a').click()
time.sleep(2)


In [5]:
no22 = 2    
while no22 == 2 : 
    try : #try 문은 예외 일어날 상황만 간략하게 적자. 
        driver.find_element_by_class_name('alex_more').click()
        time.sleep(2)
    except NoSuchElementException : 
        html0 = driver.page_source 
        soup0 = BeautifulSoup(html0, 'html.parser')
        div_list0 = soup0.find('ul', class_='list_comment').find_all('div', class_='cmt_info')
        print('%s 건 댓글 수집 요청하셨으나, %s 건 밖에 존재하지 않아서 %s 건 만큼만 수집하겠습니다' %(cnt, len(div_list0), len(div_list0)))
        cnt = len(div_list0)
        break
    else : 
        for c in range(1,3) : 
            driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)
        html0 = driver.page_source 
        soup0 = BeautifulSoup(html0, 'html.parser')
        div_list0 = soup0.find('ul', class_='list_comment').find_all('div', class_='cmt_info')
        if len(div_list0) >= cnt : 
            break          

In [6]:
nick1 = []
cont1 = []
date1 = []
go1 = []
bgo1 = []
no11 = 1

f = open(f_name, 'a', encoding='utf-8')
for a in div_list0 : 
    print('%s 번째 댓글 수집 중==================' %no11)
    f.write('%s 번째 댓글 수집 중==============' %no11 + '\n')
    
    nick = a.find('a', class_='link_nick').get_text().replace('\n', "").strip()
    print('1.작성자ID :', nick)
    f.write('1.작성자ID :'+nick+'\n')
    nick1.append(nick)
    
    try : 
        review = a.find('p', class_='desc_txt font_size_17').get_text().replace('\n', "").strip()
        print('2.댓글내용 :',review)
        f.write('2.댓글내용 :'+review+'\n')
        cont1.append(review)
    except : 
        try : 
            review = a.find('div', class_='covered_msg').get_text().replace('\n', "").strip()
            print('2.댓글내용 :',review)
            f.write('2.댓글내용 :'+review+'\n')
            cont1.append(review)
        except : 
            f.write('2.댓글내용 : 작성자가 삭제한 댓글 입니다'+'\n')
            print('2. 댓글내용 : 작성자가 삭제한 댓글 입니다')
            cont1.append("작성자가 삭제한 댓글 입니다")
    
    date = a.find('span', class_='txt_date').get_text().replace('\n', "").strip()
    print('3.작성일자 :',date)
    f.write('3.작성일자 :'+date+'\n')
    date1.append(date)
    
    try : 
        gogam = a.find('button', class_='btn_g btn_recomm #like ?c_title=%EB%8C%93%EA%B8%80%EC%B0%AC%EC%84%B1').find('span', 'num_txt').get_text().replace('\n', "").strip()
        print('4.공감횟수 :',gogam)
        f.write('4.공감횟수 :'+gogam+'\n')
        go1.append(gogam)
    except : 
        f.write('4. 공감횟수 : 0'+'\n')
        print('4. 공감횟수 : 0')
        go1.append('0')
    
    try : 
        bgogam = a.find('button', class_='btn_g btn_oppose #dislike ?c_title=%EB%8C%93%EA%B8%80%EB%B0%98%EB%8C%80').find('span', 'num_txt').get_text().replace('\n', "").strip()
        print('5.비공감횟수 : ',bgogam)
        f.write('5.비공감횟수 :'+bgogam+'\n')
        bgo1.append(bgogam)
    except : 
        f.write('5.비공감횟수 : 0'+'\n')
        print('5.비공감횟수 : 0')
        bgo1.append('0')
    
    f.write('%s 건 완료 =====================' %no11 + '\n')
    f.write('\n')
    print('%s 건 완료 =================' %no11)
    print('\n')
    
    no11 += 1 
    if no11 > cnt : 
        break
            
driver.quit()
f.close()
e_time = time.time()

1 번째 댓글 수집 중==================
1.작성자ID : 코비
2.댓글내용 : 댓글이 없네  신기하네정부 코로나대응 못 하다고 외치던  목소리 다 어디갔나?
3.작성일자 : 3시간전
4.공감횟수 : 658
5.비공감횟수 :  14
1 건 완료 =================


2 번째 댓글 수집 중==================
1.작성자ID : 김형성
2.댓글내용 : 일본은 대한민국 적폐언론들이 물고 빨고할때부터 저꼴날줄알았다 대한민국은 OECD국가중 10만명당 확진자 비율이 세번째로 낮다 그것도 인구460만인 뉴질랜드와 인구밀도 비교도 안되는 호주다음이고 기온낮은 북반구 나라중에는 단연코 가장 낮은비율이다 또 경제성장률도 계속1위이다 OECD도 빨개이냐? 전라도냐? 조선족이냐? 수준차이나는것들아 툭하면 부모님 고향까지 바꾸어놓는것들아 그리고 우리나라 코로나 확진자중에 사망자 비율은 60대이상이 95프로다 코로나가ㅇ별것아니다 독감수준이다 가짜뉴스등좀 그만보고 상식적으로들좀 살자
3.작성일자 : 3시간전
4.공감횟수 : 627
5.비공감횟수 :  11
2 건 완료 =================


3 번째 댓글 수집 중==================
1.작성자ID : 밀알처럼
2.댓글내용 : 문재인 정부 고마워 해야 한다
3.작성일자 : 3시간전
4.공감횟수 : 478
5.비공감횟수 :  39
3 건 완료 =================


4 번째 댓글 수집 중==================
1.작성자ID : 밍구
2.댓글내용 : 일본과 똑같이 해야 된다고 언론들이그렇게 난리 직이드만 이제는 걱정되냐??ㅋㅋㅋㅋㅋㅋㅋ.
3.작성일자 : 3시간전
4.공감횟수 : 326
5.비공감횟수 :  6
4 건 완료 =================


5 번째 댓글 수집 중==================
1.작성자ID : jeju man
2.댓글내용 : 조중동과 선동질 광화문패거리  다들 어디갔냐?부끄러움을 모르는 짐승들아  니네 나라로 가라.
3

In [7]:
total_time = e_time - s_time 

news = pd.DataFrame()
news['작성자ID'] = nick1
news['작성내용'] = cont1
news['작성일자'] = date1
news['공감횟수'] = go1
news['비공감횟수'] = bgo1

news.to_csv(fc_name, encoding='utf-8-sig')
news.to_excel(fx_name)

print('='*100)
print('총 %s 건 댓글 수집했습니다' %cnt)
print('총 소요시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)

총 100 건 댓글 수집했습니다
총 소요시간은 82.4 초 입니다


118